# Task 2.2 Visualize Quantum Measurements

There are different  ways to visualize measurments using Qiskit, it is important to use proper visualizations to represnt the results accuratley

More information can be found in 
https://quantum.cloud.ibm.com/docs/en/api/qiskit/visualization


## Objective 1: Classical Feedforward and Control Flow

Classical feedforward allows quantum circuits to make decisions based on measurement results, enabling adaptive quantum algorithms where later operations depend on earlier measurement outcomes.

### IF Statement with Classical Conditions

The `if_test` method allows conditional operations based on classical register values. This is essential for error correction, measurement-based quantum computing, and adaptive algorithms.

In [ ]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
 
# Create quantum and classical registers
qubits = QuantumRegister(1)
clbits = ClassicalRegister(1)
circuit = QuantumCircuit(qubits, clbits)
(q0,) = qubits
(c0,) = clbits
 
# Create superposition and measure
circuit.h(q0)                    # Create superposition |0⟩ → (|0⟩ + |1⟩)/√2
circuit.measure(q0, c0)          # Measure qubit to classical bit
 
# Conditional operation: Apply X gate ONLY if classical bit c0 equals 1
with circuit.if_test((c0, 1)):
    circuit.x(q0)                # This X gate is conditional on measurement result
 
# Measure again to see the effect
circuit.measure(q0, c0)
 
print("Circuit with classical conditional (if_test):")
circuit.draw("mpl")
 
# Expected behavior:
# - If first measurement gives |0⟩ (c0=0), no X gate applied, second measurement gives |0⟩
# - If first measurement gives |1⟩ (c0=1), X gate flips |1⟩ to |0⟩, second measurement gives |0⟩
# Final result: Always measures |0⟩ in the second measurement

### Advanced Classical Expressions

Qiskit supports complex classical expressions using the `expr` module, enabling sophisticated conditional logic based on multiple classical bits and their relationships.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.classical import expr
 
# Create a quantum error correction inspired circuit
num_qubits = 8
if num_qubits % 2 or num_qubits < 4:
    raise ValueError("num_qubits must be an even integer ≥ 4")
    
meas_qubits = list(range(2, num_qubits, 2))  # Select qubits to measure and reset
 
# Create registers
qr = QuantumRegister(num_qubits, "qr")
mr = ClassicalRegister(len(meas_qubits), "m")
qc = QuantumCircuit(qr, mr)
 
# Step 1: Create local Bell pairs (entangled states)
qc.reset(qr)                    # Initialize all qubits to |0⟩
qc.h(qr[::2])                   # Apply Hadamard to even-indexed qubits
for ctrl in range(0, num_qubits, 2):
    qc.cx(qr[ctrl], qr[ctrl + 1])  # Create Bell pairs: (|00⟩ + |11⟩)/√2
 
# Step 2: Create entanglement between neighboring Bell pairs
for ctrl in range(1, num_qubits - 1, 2):
    qc.cx(qr[ctrl], qr[ctrl + 1])
 
# Step 3: Measure boundary qubits and reset them
for k, q in enumerate(meas_qubits):
    qc.measure(qr[q], mr[k])    # Measure and store result in classical register
    qc.reset(qr[q])             # Reset measured qubit to |0⟩
 
# Step 4: Parity-conditioned X corrections (classical feedforward)
# Each non-measured qubit gets flipped if the XOR parity of preceding measurements is 1
for tgt in range(num_qubits):
    if tgt in meas_qubits:      # Skip the qubits that were measured
        continue
        
    # Find all measurement registers whose physical qubit index < current target
    left_bits = [k for k, q in enumerate(meas_qubits) if q < tgt]
    if not left_bits:           # Skip if no preceding measurements
        continue
 
    # Build XOR-parity expression using classical expressions
    parity = expr.lift(mr[left_bits[0]])  # Convert first classical bit to Value object
    for k in left_bits[1:]:
        parity = expr.bit_xor(mr[k], parity)  # Calculate cumulative XOR parity
        
    # Apply conditional X gate based on parity calculation
    with qc.if_test(parity):    # If parity == 1, apply X gate
        qc.x(qr[tgt])
 
# Step 5: Re-entangle the measured and reset qubits
for ctrl in range(1, num_qubits - 1, 2):
    qc.cx(qr[ctrl], qr[ctrl + 1])

print("Advanced circuit with classical expressions and parity corrections:")
qc.draw('mpl')

### Limitations of Classical Control Flow

Important constraints when using classical conditionals in Qiskit:

* **Bit Limit**: Operands in `if_test` statements must be 32 or fewer bits
* **Broadcasting**: Qiskit Runtime cannot broadcast more than 60 bits at a time (use barriers to manage different broadcasts)
* **No Nesting**: Nested conditionals (if_test inside another if_test) are not supported
* **No Reset/Measure in Conditionals**: Reset or measurement operations inside conditionals are not supported
* **No Arithmetic**: Arithmetic operations within classical expressions are not supported

These limitations are important to consider when designing complex classical feedforward circuits.

## Objective 2: Measurement Result Visualizations

Qiskit provides comprehensive visualization tools for analyzing quantum measurement results and quantum states.

### Basic Histogram Plotting

`plot_histogram` is the most common visualization for quantum measurement results, showing the frequency distribution of measurement outcomes.

In [ ]:
from qiskit.visualization import plot_histogram
 
# Simulated measurement results from quantum circuits
counts1 = {'00': 499, '11': 501}  # First execution: nearly equal distribution
counts2 = {'00': 511, '11': 489}  # Second execution: slight bias toward |00⟩
 
data = [counts1, counts2]
plot_histogram(data)

# This shows the probability distribution of measuring |00⟩ and |11⟩ states
# from two different circuit executions

### Customized Histogram Visualization

Common customization parameters for histograms:
- `title`: Chart title
- `figsize`: Figure dimensions (width, height) in inches
- `legend`: Labels for different data series
- `color`: Custom colors for each data series

In [ ]:
legend = ['First execution', 'Second execution']
title = 'Bell State Measurement Results'
figsize = (5, 3)
color = ['crimson', 'midnightblue']

plot_histogram(data, 
               legend=legend, 
               title=title, 
               figsize=figsize, 
               color=color)

# Customizations make plots more informative for presentations and publications

### Types of Quantum Visualizations

**Counts Visualizations:**
- `plot_histogram`: Bar chart of measurement outcome frequencies
- `plot_distribution`: Probability distribution visualization

**State Visualizations:**
- `plot_bloch_vector`: Single qubit state on Bloch sphere
- `plot_bloch_multivector`: Multiple qubits on separate Bloch spheres
- `plot_state_city`: 3D cityscape plot of statevector amplitudes
- `plot_state_hinton`: Hinton diagram for density matrix visualization
- `plot_state_paulivec`: Pauli expectation values visualization
- `plot_state_qsphere`: Q-sphere representation of quantum states

### Basic Counts Visualization

Simple histogram for a single set of measurement results.

In [ ]:
from qiskit.visualization import plot_histogram
 
# Ideal Bell state measurement results
counts = {"00": 501, "11": 499}
plot_histogram(counts)

# This represents near-perfect Bell state (|00⟩ + |11⟩)/√2 measurements
# with approximately equal probability for |00⟩ and |11⟩ outcomes

### Quantum State Visualization

Visualizing quantum states directly using city plot representation.

In [ ]:
from qiskit.visualization import plot_state_city
 
# Define a quantum state (density matrix or statevector)
# This represents a single qubit state with specific amplitudes
state = [[ 0.75  , 0.433j],    # |0⟩⟨0| and |0⟩⟨1| components
         [-0.433j, 0.25  ]]    # |1⟩⟨0| and |1⟩⟨1| components
         
plot_state_city(state, title="Quantum State City Plot")

# City plots show the magnitude of statevector or density matrix elements
# Real parts are shown in blue, imaginary parts in red

### Other Visualization Categories

**Device Visualizations:**
- `plot_gate_map`: Physical qubit connectivity and layout
- `plot_error_map`: Quantum error rates across device
- `plot_circuit_layout`: How circuits map to physical hardware
- `plot_coupling_map`: Qubit connectivity graph

**Circuit Visualizations:**
- `circuit_drawer`: Quantum circuit diagrams

**DAG Visualizations:**
- `dag_drawer`: Directed Acyclic Graph representation

**Pass Manager Visualizations:**
- `pass_manager_drawer`: Compilation pass sequences

**Timeline Visualizations:**
- `timeline_drawer`: Quantum operation scheduling

**Single Qubit State Transitions:**
- `visualize_transition`: Animated Bloch sphere transitions

**Array/Matrix Visualizations:**
- `array_to_latex`: Convert arrays to LaTeX format

## Objective 3: Gate Map Visualization

Gate map visualization shows the physical layout and connectivity of quantum processors, essential for understanding hardware constraints and optimizing circuit placement.

In [ ]:
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.visualization import plot_gate_map
 
# Create a simulated quantum backend with 5 qubits
backend = GenericBackendV2(num_qubits=5)
 
# Visualize the gate map (qubit connectivity)
plot_gate_map(backend, figsize=[3, 3], title="5-Qubit Processor Gate Map")

# The gate map shows:
# - Qubit positions (circles with numbers)
# - Connectivity (lines between qubits)
# - This helps understand which qubit pairs can directly interact via 2-qubit gates

## Summary

This notebook covered three key areas of quantum circuit visualization and control:

**1. Classical Feedforward and Control Flow:**
- `if_test` statements for conditional quantum operations
- Classical expressions with `expr` module for complex logic
- Applications in error correction and adaptive algorithms
- Important limitations and constraints

**2. Measurement Result Visualizations:**
- `plot_histogram` for measurement statistics
- Customization options for professional presentations
- Various state visualization methods (Bloch sphere, city plots, etc.)
- Multiple visualization categories for different analysis needs

**3. Hardware Visualization:**
- `plot_gate_map` for understanding quantum processor architecture
- Essential for circuit optimization and hardware-aware programming

These tools are crucial for developing, debugging, and presenting quantum algorithms, especially those requiring classical feedback and adaptive operations.